![](../images/rivacon_frontmark_combined_header.png)

# Equity Volatility Surfaces

In [1]:
# import libs
import matplotlib
matplotlib.use('nbagg')
import rivapy
from rivapy import marketdata as mkt_data
from rivapy import enums as enums
# import pyvacon
import datetime as dt
from dateutil.relativedelta import relativedelta
import math
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import plotly.graph_objects as go
import ipywidgets as widgets
from scipy.optimize import least_squares
#the next lin is a jupyter internal command to show the matplotlib graphs within the notebook
#%matplotlib inline
#reload modules
%load_ext autoreload
%autoreload 2

## General Remarks
An equity volatility surface is an object providing for arbitrary expiries and strikes implied volatilities. The volatility surfaces provided by the analytics library are parametrized w.r.t. the so-called X-strikes, i.e. one has to put in a strike w.r.t. the X-variable which is the driving process of the spot $S$, i.e.
$$ S_t=(F_t-D_t)X_t+D_t$$
where $F_t$ is the risky forward and $D_t$ the cash dividends, see [Buehler](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1141877) for a more detailed discussion. 

To create a volatility surface one needs two components:
- A reference forward curve (typically the forward curve which was used to compute the implieds from quoted prices)
- A volatility parametrization



## Creating Forward Curve
We create a dummy forward curve as shown in the  [forward_curve](equity_forwardcurve.ipynb) notebook which will be used in all subsequent volatility surface constructions.

In [8]:
refdate = dt.datetime(2017,1,1)

#dividend table neede fo forward curve
object_id = "TEST_DIV" 
ex_dates = [dt.datetime(2018,3,29), dt.datetime(2019,3,29), dt.datetime(2020,3,29), dt.datetime(2021,3,29)]
pay_dates = [dt.datetime(2018,4,1), dt.datetime(2019,4,1), dt.datetime(2020,4,1), dt.datetime(2021,4,1)]
tax_factors = [1.0, 1.0, 1.0, 1.0]
div_yield = [0, 0.005, 0.01, 0.01]
div_cash = [3.0, 2.0, 1.0, 0.0]
div_table = rivapy.marketdata.DividendTable(object_id, refdate, ex_dates, pay_dates, div_yield, div_cash, tax_factors)

#discount- and borrowing curve needed for forward curve
dates = [refdate + dt.timedelta(days=x) for x in [0,10]]
df = [1.0,1.0]
dc = mkt_data.DiscountCurve(object_id, refdate, dates, df, 
                             enums.InterpolationType.HAGAN_DF, enums.ExtrapolationType.NONE, enums.DayCounterType.Act365Fixed)
bc = mkt_data.DiscountCurve(object_id, refdate, dates, df, 
                             enums.InterpolationType.HAGAN_DF, enums.ExtrapolationType.NONE, enums.DayCounterType.Act365Fixed)
spot = 100.0

#forward curve
forward_curve = mkt_data.EquityForwardCurve(spot, dc, bc, div_table)

## Volatility Parametrizations

The volatility parametrization provides the method *calc_implied_vol* functionality to retrieve for each x-strike and time-to-maturity (in year fractions) the implied volatility. This method is used internally in the volatility surface in all methods where a implied volatility is computed.
We will discuss the different available parametrizations in this subsection.

### Flat Volatility
To setup a flat volatility, one may use the VolatilityParametrizationFlat

In [9]:
flat_param = mkt_data.VolatilityParametrizationFlat(0.3)

### Term Structure Volatility
To create a volatility which has only a term structure and no strike dependency one may use this parametrization type. This parametrization needs a vector of expiry times (given as year fractions which are interpreted as year fractions w.r.t. the day counter specified in th volatility surface) and forward at-the-money volatilities, i.e. X-strike=1.0.

In [10]:
ttm = [1.0/12.0, 1.0, 2.0, 3.0]
fwd_atm_vols =  [0.3, 0.28, 0.25, 0.24]
term_param = mkt_data.VolatilityParametrizationTerm(ttm,fwd_atm_vols)

### SSVI
This parametrization is inspired by the volatility structure provided by stochastic volatility models. The total variance $w(k,t)$ for a strike log strike $k$ and time-to-maturity $t$ is given by
$$ w(k,t) = \frac{\theta_t}{2}\left( 1+\rho\phi(\theta_t)k+\sqrt{(\phi(\theta_t)k+\rho)^2+(1-\rho^2)}  \right) $$ 
and 
$$ \phi(\theta_t) = \frac{\eta}{\theta_t^\gamma(1+\theta_t)^{1-\gamma}} $$
for parameters $\rho$, $\eta$, $\gamma$ and given atm implied total variances $\theta_t:=\sigma^2(t)t$. The term structure of implied total variances is internally approximated by interpolation from given atm volatilities. The nice property of this surface is that there are very simple conditions on the parameters to guarantee that the surface is free of arbitrage, see [gatheral_jacquier_svi](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2033323).

In [11]:
gamma = 0.5 # responsible for the "rate of decay"
rho = -0.7 # responsible for the skewness of the vol_surface
eta = 1.0 # responsible for the curvature
ssvi_param = mkt_data.VolatilityParametrizationSSVI(ttm, fwd_atm_vols, rho, eta, gamma)

In [12]:
ssvi_param.calc_implied_vol(1,.9)

0.3151536735681129

### SVI

In [46]:
svi_param = mkt_data.VolatilityParametrizationSVI(expiries=np.array([1.0/365.0, 1.0]), svi_params=[
        (0.0001, 0.1, -0.5, 0.0, 0.0001),
        (0.0002, 0.1, -0.5, 0.0, 0.00004),
])

In [47]:
svi_param.calc_implied_vol(1/365,1)

0.20037464909513877

### SABR

The [SABR](https://bsic.it/sabr-stochastic-volatility-model-volatility-smile/) model assumes that the forward rate and the instantaneous volatility are driven by two correlated Brownian motions:

$$df_t = \alpha_t f_t^\beta d W_t^1$$

$$d\alpha_t = \nu\alpha_t d W_t^2$$

$$E\bigl[d W_t^1 d W_T^2\bigr] = \rho d t$$

The expression that the implied volatility must satisfy is

$$\sigma_B(K,f) = \frac{\alpha\biggl\{1+\biggl[\frac{(1-\beta)^2}{24}\frac{\alpha^2}{(fK)^{1-\beta}}+\frac{1}{4}\frac{\rho\beta\nu\alpha}{(FK)^{(1-\beta)/2}}+\frac{2-3\rho^2}{24}\nu^2\biggr]T\biggr\}}{(fK)^{(1-\beta)/2}\biggl[1+\frac{(1-\beta)^2}{24}{ln}^2\frac{f}{K}+\frac{(1-\beta)^4}{1920}{ln}^4\frac{f}{K}\biggr]}\frac{z}{\chi(z)}$$

$$z=\frac{\nu}{\alpha}(fK)^{(1-\beta)/2}ln\frac{f}{K}$$

$$\chi(z) = ln\Biggl[\frac{\sqrt{1-2\rho z+z^2}+z-\rho}{1-\rho}\Biggr]$$

When $f = K $ (for ATM options), the above formula for implied volatility simplifies to:

$$\sigma_{ATM} = \sigma_B(f,f)=\frac{\alpha\biggl\{1+\biggl[\frac{(1-\beta)^2}{24}\frac{\alpha^2}{f^{2-2\beta}}+\frac{1}{4}\frac{\rho\beta\nu\alpha}{f^{1-\beta}}\frac{2-3\rho^2}{24}\nu^2\biggr]T\biggr\}}{f^{1-\beta}}$$

where

> $\alpha$ is the instantaneous vol;

> $\nu$ is the vol of vol;

> $\rho$ is the correlation between the Brownian motions driving the forward rate and the instantaneous vol;

> $\beta$ is the CEV component for forward rate (determines shape of forward rates, leverage effect and backbond of ATM vol).

For details please refer to the [SABR](../models/sabr_volatility_model.ipynb) notebook. 

In [15]:
# Plotting SABR Volatility Smile– Rivapy

# create widgets
style = {'description_width': 'initial'}

FloatTextAlpha = widgets.FloatText(value = 0.3, step = 0.01, description = 'Alpha')

FloatSliderNu = widgets.FloatSlider(value = 0.1, min = 0.0001, max = 5, step = 0.01, description = 'Nu',
                                       continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f')

FloatSliderBeta = widgets.FloatSlider(value = 0.1, min = 0, max = 1, step = 0.01, description = 'Beta',
                                       continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f')

FloatSliderRho = widgets.FloatSlider(value = 0.1, min = -0.999999, max = 0.999999, step = 0.01, description = 'Rho',
                                       continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f')

FloatRangeSliderStrikes = widgets.FloatRangeSlider(value=[.4, 1.6], min=0, max=3.0, step=0.05, description='Strike Range:', 
                                                   disabled=False, continuous_update=False,orientation='horizontal',
                                                   readout=True, readout_format='.1f',style=style)

FloatSliderExpiries = widgets.FloatSlider(value=1, min=1/12, max=30.0, step=1/12, description='Expiry:', 
                                                   disabled=False, continuous_update=False,orientation='horizontal',
                                                   readout=True, readout_format='.2f',style=style)

ButtonNewPlot = widgets.Button(description="New Plot")

ButtonAddTrace = widgets.Button(description="Add Trace")

global OutputWidget
OutputWidget = widgets.Output()

def create_vol_grid(alpha, nu, beta, rho,strike_range, expiry):
    F_0 = 1
    strikes = np.linspace(strike_range[0], strike_range[1], num=100)
    sabr_params = np.array([[alpha,nu,beta,rho]])
    ttm = [expiry]
    sabr_param = mkt_data.VolatilityParametrizationSABR(ttm, sabr_params)
    vols = [sabr_param.calc_implied_vol(expiry,x) for x in strikes]

    return strikes, vols

def create_plot(strikes, vols,expiry,alpha, nu, beta, rho):    
    fig.add_trace(go.Scatter(x= strikes,y= vols 
                            ,mode = 'lines+markers'
                          ,hovertemplate = 
                            'Moneyness:  %{x: .1%}' #+\
                            +'<br>Volatility: %{y: .1%}'
                            +'<br>Expiry: {:,.0f} Yrs'.format(expiry)
                            +'<br>Alpha: {:,.1%}'.format(alpha)
                            +'<br>Nu: {:,.1f}'.format(nu)
                            +'<br>Beta: {:,.1f}'.format(beta)
                            +'<br>Rho: {:,.1f}'.format(rho)
                            +'<extra></extra>',
                            showlegend=False)
                         )


    fig.update_layout(title={
                          'text': "<b>Volatility Smile</b>",
                          'y':0.95,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'
                            }
                    ,width=1000
                    ,height=500
                    ,xaxis_title='Moneyness'
                    ,xaxis_tickformat = '.1%'
                    ,xaxis_range=[strikes.min(),strikes.max()]
                    ,yaxis_title='Volatility'
                    ,yaxis_tickformat = '.1%'
                    ,yaxis_range=[0,1]
                    ,font=dict(
                      family="Courier New, monospace"
                      ,size=10
                      )
                    ,margin=dict(l=65, r=50, b=65, t=90)
    )
    fig.show()

def plot(alpha, nu, beta, rho,strike_range, expiry):
#     function is called by eventhandler, i.e. if input parameter changes

#     clear output
    OutputWidget.clear_output()
    
#     1. create vol grid
    strikes, vols = create_vol_grid(alpha, nu, beta, rho,strike_range, expiry)

#     2. plot surface
    create_plot(strikes, vols,expiry,alpha, nu, beta, rho)
    

def eventhandler(change):
          
    alpha = FloatTextAlpha.value
    nu = FloatSliderNu.value
    beta = FloatSliderBeta.value
    rho = FloatSliderRho.value
    strike_range = FloatRangeSliderStrikes.value
    expiry = FloatSliderExpiries.value
    
#     call plot function
    with OutputWidget:
        plot(alpha, nu, beta, rho,strike_range, expiry)
        
def eventhandler2(change):
    global fig
    
    fig = go.Figure()
    
    alpha = FloatTextAlpha.value
    nu = FloatSliderNu.value
    beta = FloatSliderBeta.value
    rho = FloatSliderRho.value
    strike_range = FloatRangeSliderStrikes.value
    expiry = FloatSliderExpiries.value
    
    with OutputWidget:
        plot(alpha, nu, beta, rho,strike_range, expiry)

# bind eventhandler to widgets
ButtonAddTrace.on_click(eventhandler)
ButtonNewPlot.on_click(eventhandler2)

# widgets groups
WidgetsGrpH1 = widgets.HBox(children=[widgets.Label('Set Chart Area:')])
WidgetsGrpH2 = widgets.HBox(children=[FloatRangeSliderStrikes])
WidgetsGrpH3 = widgets.HBox(children=[widgets.Label('Set Parameters:')])
WidgetsGrpH4 = widgets.HBox(children=[FloatTextAlpha,FloatSliderExpiries])
WidgetsGrpH5 = widgets.HBox(children=[FloatSliderNu,FloatSliderBeta,FloatSliderRho])
WidgetsGrpH6 = widgets.HBox(children=[ButtonNewPlot,ButtonAddTrace])
WidgetsGrpV1 = widgets.VBox(children=[WidgetsGrpH1,WidgetsGrpH2,WidgetsGrpH3,WidgetsGrpH4,WidgetsGrpH5,WidgetsGrpH6])

display(WidgetsGrpV1)
display(OutputWidget)

Output()

In [16]:
ttm = [1.0/12.0, 1.0, 2.0, 3.0]
sabr_params = np.array([[.1, 0.1, .9,-.8],
                        [.3, 0.1, .1, .1],
                        [.5, .3, .9, -.75,],
                        [.5, .3, .9, -.85,]])

In [17]:
sabr_param = mkt_data.VolatilityParametrizationSABR(ttm, sabr_params)
sabr_param.calc_implied_vol(1,1)

0.30118

## Volatility Smile

### Plot Parametrization

In [18]:
vol_params_dict = {'Flat': flat_param
                   ,'Term': term_param
                   ,'SVI': svi_param
                   ,'SSVI': ssvi_param
                   ,'SABR': sabr_param
                  }

In [19]:
# Comparing Volatility Smiles – Rivapy

# create widgets
style = {'description_width': 'initial'}

keys_list = list(vol_params_dict.keys())
SelectMultipleParams = widgets.SelectMultiple(options=keys_list,value=[keys_list[0]],description='Parametrization',
                                                disabled=False, continuous_update=False,readout=True,style=style)

FloatRangeSliderStrikes = widgets.FloatRangeSlider(value=[.4, 1.6], min=0, max=3.0, step=0.05, description='Strike Range', 
                                                   disabled=False, continuous_update=False,orientation='horizontal',
                                                   readout=True, readout_format='.1f',style=style)

DatePickerRefDate = widgets.DatePicker(description='Ref Date',value = dt.date(2017,1,1), disabled=False, continuous_update=False,
                                                   readout=True,style=style)

DatePickerExpiryDate = widgets.DatePicker(description='Expiry Date',value = dt.date(2018,1,1), disabled=False, continuous_update=False,
                                                   readout=True,style=style)


ButtonNewPlot = widgets.Button(description="New Plot")

ButtonAddTrace = widgets.Button(description="Add Trace(s)")

global OutputWidget
OutputWidget = widgets.Output()


def create_plot(param_list,strike_range, ref_date, expiry_date):    
    
    moneyness = np.linspace(strike_range[0], strike_range[1], num=100)
    strikes = np.array([i*forward_curve.value(ref_date,expiry_date) for i in moneyness])
    expiry = (expiry_date-ref_date).days/365.25
    
    for p in param_list:
        param = vol_params_dict[p]
        vol_surf = mkt_data.VolatilitySurface(p+'_surf', ref_date, forward_curve, enums.DayCounterType.Act365Fixed, param)
        vols = [vol_surf.calc_implied_vol(expiry_date,x,ref_date) for x in strikes]        
    
        fig_smile.add_trace(go.Scatter(x= moneyness,y= vols 
                                ,mode = 'lines+markers'
                              ,hovertemplate =
                                '<br>Parametrization: {}'.format(p)
                                +'<br>Moneyness:  %{x: .1%}' #+\
                                +'<br>Volatility: %{y: .1%}'
                                +'<br>Expiry: {:,.2f} Yrs'.format(expiry)
                                +'<extra></extra>',
                                showlegend=False)
                             )


    fig_smile.update_layout(title={
                          'text': "<b>Volatility Smile</b>",
                          'y':0.95,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'
                            }
                    ,width=1000
                    ,height=500
                    ,xaxis_title='Moneyness'
                    ,xaxis_tickformat = '.1%'
                    ,xaxis_range=[moneyness.min(),moneyness.max()]
                    ,yaxis_title='Volatility'
                    ,yaxis_tickformat = '.1%'
                    ,yaxis_range=[0,1]
                    ,font=dict(
                      family="Courier New, monospace"
                      ,size=10
                      )
                    ,margin=dict(l=65, r=50, b=65, t=90)
    )
    fig_smile.show()

def plot(param_list,strike_range, ref_date, expiry_date):
#     function is called by eventhandler, i.e. if input parameter changes

#     clear output
    OutputWidget.clear_output()

#     plot surface
    create_plot(param_list,strike_range, ref_date, expiry_date)
    

def eventhandler(change):
          
    param_list = list(SelectMultipleParams.value)
    strike_range = FloatRangeSliderStrikes.value
    ref_date = dt.datetime.combine(DatePickerRefDate.value, dt.datetime.min.time())
    expiry_date = dt.datetime.combine(DatePickerExpiryDate.value, dt.datetime.min.time())
    
#     call plot function
    with OutputWidget:
        plot(param_list,strike_range, ref_date, expiry_date)
        
def eventhandler2(change):
    global fig_smile
    
    fig_smile = go.Figure()
    
    param_list = list(SelectMultipleParams.value)
    strike_range = FloatRangeSliderStrikes.value
    ref_date = dt.datetime.combine(DatePickerRefDate.value, dt.datetime.min.time())
    expiry_date = dt.datetime.combine(DatePickerExpiryDate.value, dt.datetime.min.time())
    
    with OutputWidget:
        plot(param_list,strike_range, ref_date, expiry_date)

# bind eventhandler to widgets
ButtonAddTrace.on_click(eventhandler)
ButtonNewPlot.on_click(eventhandler2)

# widgets groups
WidgetsGrpV1 = widgets.VBox(children=[widgets.Label('Set Chart Area:'),FloatRangeSliderStrikes])
WidgetsGrpV2 = widgets.VBox(children=[widgets.Label('Set Reference and Expiry Date:'),DatePickerRefDate,DatePickerExpiryDate])
WidgetsGrpV3 = widgets.VBox(children=[widgets.Label('Select Parametrization(s):'),SelectMultipleParams])
WidgetsGrpH1 = widgets.HBox(children=[WidgetsGrpV1,WidgetsGrpV2,WidgetsGrpV3])
WidgetsGrpH2 = widgets.HBox(children=[ButtonNewPlot,ButtonAddTrace])

display(WidgetsGrpH1,WidgetsGrpH2)
display(OutputWidget)

Output()

### Fitting Parametrizations

In [82]:
# custruct equity option quote table
# data = {'Expiry': [1, 1, 1, 1],'Strike': [0.8, 0.9, 1.0, 1.1], 'Bid': [.23,.235,.24,.245], 'Ask': [.24,.245,.25,.255]}
data = {'Expiry': [1, 1, 1, 1, 1.5, 1.5, 1.5, 1.5],
        'Strike': [0.8, 0.9, 1.0, 1.1,0.8, 0.9, 1.0, 1.1], 
        'Bid': [.23,.235,.24,.245,.24,.245,.25,.255], 
        'Ask': [.24,.245,.25,.255,.25,.255,.26,.265]}
df_quote_table = pd.DataFrame(data)
df_quote_table

,Expiry,Strike,Bid,Ask
0,1.0,0.8,0.230,0.240
1,1.0,0.9,0.235,0.245
2,1.0,1.0,0.240,0.250
3,1.0,1.1,0.245,0.255
4,1.5,0.8,0.240,0.250
5,1.5,0.9,0.245,0.255
6,1.5,1.0,0.250,0.260
7,1.5,1.1,0.255,0.265


In [32]:
strikes = np.array([i*forward_curve.value(refdate,refdate+relativedelta(years=1)) for i in df_quote_table.Strike.tolist()])
# ttm = [1]
# sabr_params = np.array([[.1, 0.1, .9,-.8]])  
ttm = [1, 1.5]
sabr_params = np.array([[.1, 0.1, .9,-.8],[.3, 0.12, .7,-.7]])  
sabr_param = mkt_data.VolatilityParametrizationSABR(ttm, sabr_params)

In [84]:
def calibrate(vol_param, quotes):
    def cost_function(x):
        vol_param._set_param(x)
        quotes['Vols'] = [vol_param.calc_implied_vol(expiry,strike) for expiry, strike in zip(quotes['Expiry'],quotes['Strike'])]
        quotes['Dist_Ask']  = [max(vol-ask,0) for ask, vol in zip(quotes['Ask'],quotes['Vols'])]
        quotes['Dist_Bid']  = [max(bid-vol,0) for bid, vol in zip(quotes['Bid'],quotes['Vols'])]
        quotes['Dist_Total'] = quotes['Dist_Ask']+quotes['Dist_Bid']
        return np.copy(quotes['Dist_Total'].values)
    
    
    result = least_squares(cost_function,vol_param._x,method='lm',diff_step=1e-3)
    
    return result.x

calibrate(sabr_param,df_quote_table)

array([ 0.24217437, -0.15138834,  0.9230154 , -0.3573693 ,  0.31364525,
        0.19898683,  1.34362096,  0.14563797])

In [54]:
sabr_param._set_param(calibrate(sabr_param,df_quote_table))

In [76]:
# plot
expiry = 1
ref_date = dt.datetime(2018,1,1)
expiry_date = ref_date + relativedelta(years=expiry)
moneyness = np.linspace(.5, 1.5, num=100)
strikes = np.array([i*forward_curve.value(ref_date,expiry_date) for i in moneyness])

df_quotes = df_quote_table[df_quote_table.Expiry == expiry].copy()
    
fig_smile = go.Figure()
sabr_param._set_param(calibrate(sabr_param,df_quote_table))
vol_surf = mkt_data.VolatilitySurface('sabr_surf', ref_date, forward_curve, enums.DayCounterType.Act365Fixed, sabr_param)
vols = [vol_surf.calc_implied_vol(expiry_date,x,ref_date) for x in strikes]        

fig_smile.add_trace(go.Scatter(x= moneyness,y= vols 
                        ,mode = 'lines+markers'
                      ,hovertemplate =
                        '<br>Parametrization: {}'.format('SABR')
                        +'<br>Moneyness:  %{x: .1%}' #+\
                        +'<br>Volatility: %{y: .1%}'
                        +'<br>Expiry: {:,.2f} Yrs'.format(expiry)
                        +'<extra></extra>',
                        showlegend=False)
                     )

fig_smile.add_trace(go.Scatter(x=df_quotes.Strike, 
                               y= df_quotes.Ask, 
                               mode = 'markers',
                               marker_symbol=106,
                               marker_color='rgb(11, 83, 148)',
                              hovertemplate = 
                                '<br>Ask Quote'
                                +'<br>Moneyness:  %{x: .1%}' #+\
                                +'<br>Volatility: %{y: .1%}'
                                +'<br>Expiry: {:,.2f} Yrs'.format(expiry)
                                +'<extra></extra>',
                              showlegend=False)
                   
                   )

fig_smile.add_trace(go.Scatter(x=df_quotes.Strike, 
                               y= df_quotes.Bid, 
                               mode = 'markers',
                               marker_symbol=105,
                               marker_color='rgb(204,0,0)',
                              hovertemplate = 
                                '<br>Bid Quote'
                                +'<br>Moneyness:  %{x: .1%}' #+\
                                +'<br>Volatility: %{y: .1%}'
                                +'<br>Expiry: {:,.2f} Yrs'.format(expiry)
                                +'<extra></extra>',
                              showlegend=False)
                   
                   )


fig_smile.update_layout(title={
                      'text': "<b>Volatility Smile</b>",
                      'y':0.95,
                      'x':0.5,
                      'xanchor': 'center',
                      'yanchor': 'top'
                        }
                ,width=1000
                ,height=500
                ,xaxis_title='Moneyness'
                ,xaxis_tickformat = '.1%'
                ,xaxis_range=[moneyness.min(),moneyness.max()]
                ,yaxis_title='Volatility'
                ,yaxis_tickformat = '.1%'
                ,yaxis_range=[.2,.3]
                ,font=dict(
                  family="Courier New, monospace"
                  ,size=10
                  )
                ,margin=dict(l=65, r=50, b=65, t=90)
)
fig_smile.show()

In [77]:
svi_param._set_param(calibrate(svi_param,df_quote_table))

ValueError: Method 'lm' doesn't work when the number of residuals is less than the number of variables.

In [81]:
ssvi_param._set_param(calibrate(ssvi_param,df_quote_table))

AttributeError: 'VolatilityParametrizationSSVI' object has no attribute '_set_param'

In [52]:
vol_surf = mkt_data.VolatilitySurface('_surf', refdate, forward_curve, enums.DayCounterType.Act365Fixed, svi_param)
vols = [vol_surf.calc_implied_vol(refdate+relativedelta(years=1),x,refdate) for x in strikes]

In [53]:
vols

[0.1899877287331567,
 0.13067704706536457,
 0.02519320525480939,
 0.07249891132249006,
 0.1899877287331567,
 0.13067704706536457,
 0.02519320525480939,
 0.07249891132249006]

In [29]:
# Comparing Volatility Smiles – Rivapy

# create widgets
style = {'description_width': 'initial'}

keys_list = list(vol_params_dict.keys())
SelectMultipleParams = widgets.SelectMultiple(options=keys_list,value=[keys_list[0]],description='Parametrization',
                                                disabled=False, continuous_update=False,readout=True,style=style)

FloatRangeSliderStrikes = widgets.FloatRangeSlider(value=[.4, 1.6], min=0, max=3.0, step=0.05, description='Strike Range', 
                                                   disabled=False, continuous_update=False,orientation='horizontal',
                                                   readout=True, readout_format='.1f',style=style)

DatePickerRefDate = widgets.DatePicker(description='Ref Date',value = dt.date(2017,1,1), disabled=False, continuous_update=False,
                                                   readout=True,style=style)

DatePickerExpiryDate = widgets.DatePicker(description='Expiry Date',value = dt.date(2018,1,1), disabled=False, continuous_update=False,
                                                   readout=True,style=style)


ButtonNewPlot = widgets.Button(description="New Plot")

ButtonAddTrace = widgets.Button(description="Add Trace(s)")

global OutputWidget
OutputWidget = widgets.Output()

def calibrate(vol_param, quotes):
    def cost_function(x):
        vol_param._set_param(x)
        quotes['Vols'] = [vol_param.calc_implied_vol(expiry,strike) for expiry, strike in zip(quotes['Expiry'],quotes['Strike'])]
        quotes['Dist_Ask']  = [max(vol-ask,0) for ask, vol in zip(quotes['Ask'],quotes['Vols'])]
        quotes['Dist_Bid']  = [max(bid-vol,0) for bid, vol in zip(quotes['Bid'],quotes['Vols'])]
        quotes['Dist_Total'] = quotes['Dist_Ask']+quotes['Dist_Bid']
        return np.copy(quotes['Dist_Total'].values)
    
    
    result = least_squares(cost_function,vol_param._x,method='lm',diff_step=1e-3)
    
    return result.x

def create_plot(param_list,strike_range, ref_date, expiry_date):    
    
    moneyness = np.linspace(strike_range[0], strike_range[1], num=100)
    strikes = np.array([i*forward_curve.value(ref_date,expiry_date) for i in moneyness])
    expiry = (expiry_date-ref_date).days/365.25
    
    for p in param_list:
        param = vol_params_dict[p]
        if p not in ['Flat','Term','SSVI']:#haben alle kein set param
            param._set_param(calibrate(sabr_param,df_quote_table))
        vol_surf = mkt_data.VolatilitySurface(p+'_surf', ref_date, forward_curve, enums.DayCounterType.Act365Fixed, param)
        vols = [vol_surf.calc_implied_vol(expiry_date,x,ref_date) for x in strikes]        
    
        fig_smile.add_trace(go.Scatter(x= moneyness,y= vols 
                                ,mode = 'lines+markers'
                              ,hovertemplate =
                                '<br>Parametrization: {}'.format(p)
                                +'<br>Moneyness:  %{x: .1%}' #+\
                                +'<br>Volatility: %{y: .1%}'
                                +'<br>Expiry: {:,.2f} Yrs'.format(expiry)
                                +'<extra></extra>',
                                showlegend=False)
                             )


    fig_smile.update_layout(title={
                          'text': "<b>Volatility Smile</b>",
                          'y':0.95,
                          'x':0.5,
                          'xanchor': 'center',
                          'yanchor': 'top'
                            }
                    ,width=1000
                    ,height=500
                    ,xaxis_title='Moneyness'
                    ,xaxis_tickformat = '.1%'
                    ,xaxis_range=[moneyness.min(),moneyness.max()]
                    ,yaxis_title='Volatility'
                    ,yaxis_tickformat = '.1%'
                    ,yaxis_range=[0,1]
                    ,font=dict(
                      family="Courier New, monospace"
                      ,size=10
                      )
                    ,margin=dict(l=65, r=50, b=65, t=90)
    )
    fig_smile.show()

def plot(param_list,strike_range, ref_date, expiry_date):
#     function is called by eventhandler, i.e. if input parameter changes

#     clear output
    OutputWidget.clear_output()

#     plot surface
    create_plot(param_list,strike_range, ref_date, expiry_date)
    

def eventhandler(change):
          
    param_list = list(SelectMultipleParams.value)
    strike_range = FloatRangeSliderStrikes.value
    ref_date = dt.datetime.combine(DatePickerRefDate.value, dt.datetime.min.time())
    expiry_date = dt.datetime.combine(DatePickerExpiryDate.value, dt.datetime.min.time())
    
#     call plot function
    with OutputWidget:
        plot(param_list,strike_range, ref_date, expiry_date)
        
def eventhandler2(change):
    global fig_smile
    
    fig_smile = go.Figure()
    
    param_list = list(SelectMultipleParams.value)
    strike_range = FloatRangeSliderStrikes.value
    ref_date = dt.datetime.combine(DatePickerRefDate.value, dt.datetime.min.time())
    expiry_date = dt.datetime.combine(DatePickerExpiryDate.value, dt.datetime.min.time())
    
    with OutputWidget:
        plot(param_list,strike_range, ref_date, expiry_date)

# bind eventhandler to widgets
ButtonAddTrace.on_click(eventhandler)
ButtonNewPlot.on_click(eventhandler2)

# widgets groups
WidgetsGrpV1 = widgets.VBox(children=[widgets.Label('Set Chart Area:'),FloatRangeSliderStrikes])
WidgetsGrpV2 = widgets.VBox(children=[widgets.Label('Set Reference and Expiry Date:'),DatePickerRefDate,DatePickerExpiryDate])
WidgetsGrpV3 = widgets.VBox(children=[widgets.Label('Select Parametrization(s):'),SelectMultipleParams])
WidgetsGrpH1 = widgets.HBox(children=[WidgetsGrpV1,WidgetsGrpV2,WidgetsGrpV3])
WidgetsGrpH2 = widgets.HBox(children=[ButtonNewPlot,ButtonAddTrace])

display(WidgetsGrpH1,WidgetsGrpH2)
display(OutputWidget)

Output()

In [ ]:
mkt_data.EquityOptionQuoteTable('Test',dt.datetime(2020,1,1,0,0,0),1.0,'Adidas','Adidas',enums.SecuritizationLevel.COLLATERALIZED,enums.Currency.EUR)

## Volatility Surface
The forward curve and the parametrization can now be combined into a VolatilitySurface

In [ ]:
obj_id = 'TEST_SURFACE'
vol_surf = mkt_data.VolatilitySurface(obj_id, refdate, forward_curve, enums.DayCounterType.Act365Fixed, sabr_param)

To compute an implied volatility, one may use the method *calcImpliedVol*. Note that this method applies a sticky-strike handling of volatilities, i.e. it assumes that the implied volatility given a certain strike is independent of current forward values which may differ from the forwards when the volatility surface was calibrated.

In [ ]:
# vol = vol_surf.calc_implied_vol(refdate,refdate + dt.timedelta(days=180),120)
vol = vol_surf.calc_implied_vol(refdate + dt.timedelta(days=180),120,refdate)
print(vol)

By executing the following command line, the volatility surface is plotted.

In [ ]:
#
refdate = dt.datetime(2017, 1, 1, 0, 0, 0)
expiries = [
    dt.datetime(2017, 2, 1, 0, 0, 0),
    dt.datetime(2018, 1, 1, 0, 0, 0),
    dt.datetime(2019, 1, 1, 0, 0, 0),
    dt.datetime(2020, 1, 1, 0, 0, 0)
]

# strikes = list(s_range(80, 120, 100))
moneyness = np.linspace(0.5, 1.5, 100)

y = moneyness
x = ttm

term_structure = []
for i in moneyness:
    temp = []
    for j in expiries:
        strike = i * forward_curve.value(refdate, j)
        temp.append(vol_surf.calc_implied_vol(j, strike, refdate))
    term_structure.append(temp)

fig = go.Figure(data=[go.Surface(x=x, y=y,z=term_structure
                        ,contours = {"x": {"show": True,"size": 0.1, "color":"red"},
                                    "y": {"show": True,"size": 0.1, "color":"red"},}
                        ,opacity = .75
                        ,hovertemplate =
                        'Moneyness:  %{y: .2%}' +\
                        '<br>Maturity (yrs): %{x: .1f}' +\
                        '<br>Volatility: %{z: .2f}<extra></extra>'
                     ,colorscale = 'temps')
                     ])

fig.update_layout(
    title={
        'text': "<b>Volatility Surface</b>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
    # ,autosize=True
    ,
    width=1000,
    height=500,
    scene=dict(xaxis_title='Maturity (yrs)',
               xaxis_tickformat='.1f',
               xaxis_autorange='reversed',
               yaxis_title='Moneyness',
               yaxis_tickformat='.2%',
               zaxis_title='Volatility',
               zaxis_tickformat='.2%'),
    font=dict(family="Courier New, monospace", size=10),
    margin=dict(l=65, r=50, b=65, t=90))

fig.show()

### Stickiness assumptions

In [ ]:
obj_id_shifted = 'TEST_SURFACE_FWD_SHIFTED'
forward_curve_shifted = mkt_data.EquityForwardCurve(1.2*spot,dc,bc,div_table)

vol_surf = mkt_data.VolatilitySurface(obj_id, refdate, forward_curve, enums.DayCounterType.Act365Fixed, sabr_param)
vol = vol_surf.calc_implied_vol(refdate + dt.timedelta(days=180),120,refdate)

mkt_data.VolatilitySurface.set_stickyness(enums.VolatilityStickyness.StickyXStrike)
vol_surf_shifted_stickyxstrike = mkt_data.VolatilitySurface(obj_id_shifted, refdate, forward_curve_shifted, enums.DayCounterType.Act365Fixed, sabr_param)
vol_shifted_stickyxstrike = vol_surf_shifted_stickyxstrike.calc_implied_vol(refdate + dt.timedelta(days=180),120,refdate)

mkt_data.VolatilitySurface.set_stickyness(enums.VolatilityStickyness.StickyStrike)
vol_surf_shifted_stickystrike = mkt_data.VolatilitySurface(obj_id_shifted, refdate, forward_curve_shifted, enums.DayCounterType.Act365Fixed, sabr_param)
vol_shifted_stickystrike = vol_surf_shifted_stickystrike.calc_implied_vol(refdate + dt.timedelta(days=180),120,refdate)

mkt_data.VolatilitySurface.set_stickyness(enums.VolatilityStickyness.StickyFwdMoneyness)
vol_surf_shifted_stickyfwdmoneyness = mkt_data.VolatilitySurface(obj_id_shifted, refdate, forward_curve_shifted, enums.DayCounterType.Act365Fixed, sabr_param)
vol_surf_shifted_stickyfwdmoneyness = vol_surf_shifted_stickyfwdmoneyness.calc_implied_vol(refdate + dt.timedelta(days=180),120,refdate)

mkt_data.VolatilitySurface.set_stickyness(enums.VolatilityStickyness.NONE)
vol_surf_shifted_none = mkt_data.VolatilitySurface(obj_id_shifted, refdate, forward_curve_shifted, enums.DayCounterType.Act365Fixed, sabr_param)
vol_surf_shifted_none = vol_surf_shifted_none.calc_implied_vol(refdate + dt.timedelta(days=180),120,refdate)

print(vol)
print(vol_shifted_stickyxstrike)
print(vol_shifted_stickystrike)
print(vol_surf_shifted_stickyfwdmoneyness)
print(vol_surf_shifted_none)